In [1]:
import random

# Elliptic Curve over a finite field: y^2 = x^3 + ax + b (mod p)
class ECC:
    def __init__(self, a, b, p, G, n):
        self.a = a  # Curve coefficient a
        self.b = b  # Curve coefficient b
        self.p = p  # Prime field
        self.G = G  # Base point (generator)
        self.n = n  # Order of the base point

    def inverse_mod(self, k):
        if k == 0:
            raise ZeroDivisionError("Cannot divide by zero")
        return pow(k, self.p - 2, self.p)

    def point_add(self, P, Q):
        if P is None:
            return Q
        if Q is None:
            return P
        if P == Q:
            return self.point_double(P)

        x1, y1 = P
        x2, y2 = Q

        if x1 == x2 and y1 != y2:
            return None  # Point at infinity

        m = ((y2 - y1) * self.inverse_mod(x2 - x1)) % self.p
        x3 = (m * m - x1 - x2) % self.p
        y3 = (m * (x1 - x3) - y1) % self.p
        return (x3, y3)

    def point_double(self, P):
        if P is None:
            return None
        x, y = P
        m = ((3 * x * x + self.a) * self.inverse_mod(2 * y)) % self.p
        x3 = (m * m - 2 * x) % self.p
        y3 = (m * (x - x3) - y) % self.p
        return (x3, y3)

    def scalar_mult(self, k, P):
        R = None
        while k:
            if k & 1:
                R = self.point_add(R, P)
            P = self.point_double(P)
            k >>= 1
        return R
a = 2
b = 3
p = 97
G = (3, 6)
n = 5

ecc = ECC(a, b, p, G, n)

private_key = random.randint(1, n - 1)
public_key = ecc.scalar_mult(private_key, G)

print("Private Key:", private_key)
print("Public Key: ", public_key)

def encrypt(ecc, G, public_key, msg_point):
    k = random.randint(1, ecc.n - 1)
    R = ecc.scalar_mult(k, G)
    S = ecc.scalar_mult(k, public_key)
    encrypted_point = ecc.point_add(msg_point, S)
    return (R, encrypted_point)

def decrypt(ecc, private_key, R, encrypted_point):
    S = ecc.scalar_mult(private_key, R)
    S_neg = (S[0], -S[1] % ecc.p)
    return ecc.point_add(encrypted_point, S_neg)

msg = (10, 20)  # Must be a valid point on the curve in practice
print('Message:', msg)
ciphertext = encrypt(ecc, G, public_key, msg)
print("Encrypted:", ciphertext)

decrypted = decrypt(ecc, private_key, *ciphertext)
print("Decrypted:", decrypted)

Private Key: 2
Public Key:  (80, 10)
Message: (10, 20)
Encrypted: ((3, 6), (9, 63))
Decrypted: (10, 20)
